In [2]:
import pandas as pd
from postal.expand import expand_address

# **DPE ADEME**

### LOGEMENTS EXISTANTS ###

In [8]:
df = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-existants.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

print(df.head())
df['Date_réception_DPE'].count()

  Date_réception_DPE Date_établissement_DPE                     Modèle_DPE  \
0         2021-12-23             2021-12-22  DPE 3CL 2021 méthode logement   
1         2022-01-03             2021-12-01  DPE 3CL 2021 méthode logement   
2         2023-04-07             2023-04-07  DPE 3CL 2021 méthode logement   
3         2021-12-22             2021-12-02  DPE 3CL 2021 méthode logement   
4         2022-01-03             2021-10-26  DPE 3CL 2021 méthode logement   

  Date_fin_validité_DPE  Version_DPE  \
0            2031-12-21          2.0   
1            2031-11-30          1.1   
2            2033-04-06          2.2   
3            2031-12-01          1.1   
4            2031-10-25          1.1   

                             Méthode_application_DPE Etiquette_DPE  \
0                         dpe appartement individuel             D   
1                         dpe appartement individuel             D   
2  dpe appartement généré à partir des données DP...             D   
3  dpe iss

4537525

On filtre pour n'avoir que les données du département 44 en 2022.

In [9]:

df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
dfv1 = df[df['Date_réception_DPE'].dt.year == 2022].copy()
dfv2 = dfv1[dfv1['N°_département_(BAN)']=='44'].copy()
print(dfv2.head())

    Date_réception_DPE Date_établissement_DPE                     Modèle_DPE  \
55          2022-10-26             2022-10-26  DPE 3CL 2021 méthode logement   
61          2022-04-05             2022-04-05  DPE 3CL 2021 méthode logement   
103         2022-05-02             2022-05-02  DPE 3CL 2021 méthode logement   
178         2022-05-10             2022-05-10  DPE 3CL 2021 méthode logement   
223         2022-05-24             2022-05-24  DPE 3CL 2021 méthode logement   

    Date_fin_validité_DPE  Version_DPE  \
55             2032-10-25          2.2   
61             2032-04-04          2.0   
103            2032-05-01          2.1   
178            2032-05-09          2.1   
223            2032-05-23          2.1   

                               Méthode_application_DPE Etiquette_DPE  \
55   dpe appartement généré à partir des données DP...             D   
61                          dpe appartement individuel             G   
103                         dpe appartement indivi

In [10]:
dfv2['Date_réception_DPE'].count()

64490

In [11]:
def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        normalized = expand_address(address)  
        return "; ".join(normalized)  
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

dfv2['Adresse_Normalisee'] = dfv2['Adresse_(BAN)'].apply(normalize_address)

print(dfv2[['Date_réception_DPE', 'Adresse_(BAN)', 'Adresse_Normalisee']].head())

    Date_réception_DPE                                  Adresse_(BAN)  \
55          2022-10-26  25 Rue anne de bretagne 44130 Fay-de-Bretagne   
61          2022-04-05         23 Rue Jean Jaurès 44600 Saint-Nazaire   
103         2022-05-02                29 Route de Vannes 44100 Nantes   
178         2022-05-10  9 Rue du Haut Moulinier 44690 Château-Thébaud   
223         2022-05-24      9 Place des Aveneaux 44800 Saint-Herblain   

                                    Adresse_Normalisee  
55   25 rue anne de bretagne 44130 fay-de-bretagne;...  
61   23 rue jean jaures 44600 saint-nazaire; 23 rue...  
103                    29 route de vannes 44100 nantes  
178  9 rue du haut moulinier 44690 chateau-thebaud;...  
223  9 place des aveneaux 44800 saint-herblain; 9 p...  


### LOGEMENTS NEUFS ###

In [12]:
df2 = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-neufs.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

print(df2.head())
df2['Date_réception_DPE'].count()

  Date_réception_DPE Date_établissement_DPE Date_fin_validité_DPE  \
0         2021-09-30             2021-09-29            2031-09-28   
1         2022-02-04             2022-02-02            2032-02-01   
2         2023-04-05             2023-04-05            2033-04-04   
3         2021-09-01             2021-09-01            2031-08-31   
4         2022-07-25             2022-07-25            2032-07-24   

   Version_DPE                  Modèle_DPE Etiquette_DPE Etiquette_GES  \
0          1.0  DPE NEUF logement : RT2012             C             C   
1          2.0  DPE NEUF logement : RT2012             B             A   
2          2.2  DPE NEUF logement : RT2012             A             A   
3          1.0  DPE NEUF logement : RT2012             C             C   
4          2.1  DPE NEUF logement : RT2012             A             A   

  Type_bâtiment  Surface_habitable_logement             Adresse_brute  ...  \
0        maison                        43.2       42 rue de Ve

537952

In [13]:
df2['Date_réception_DPE'] = pd.to_datetime(df2['Date_réception_DPE'], errors='coerce')
df2v1 = df2[df2['Date_réception_DPE'].dt.year == 2022].copy()
df2v2 = df2v1[df2v1['N°_département_(BAN)']=='44'].copy()
print(df2v2.head())

    Date_réception_DPE Date_établissement_DPE Date_fin_validité_DPE  \
103         2022-04-28             2022-04-28            2032-04-27   
124         2022-02-22             2022-02-22            2032-02-21   
418         2022-01-25             2022-01-25            2032-01-24   
613         2022-12-22             2022-12-22            2032-12-21   
814         2022-05-30             2022-05-30            2032-05-29   

     Version_DPE                  Modèle_DPE Etiquette_DPE Etiquette_GES  \
103          2.1  DPE NEUF logement : RT2012             A             A   
124          2.0  DPE NEUF logement : RT2012             A             A   
418          2.0  DPE NEUF logement : RT2012             A             A   
613          2.2  DPE NEUF logement : RT2012             A             A   
814          2.1  DPE NEUF logement : RT2012             A             A   

    Type_bâtiment  Surface_habitable_logement  \
103        maison                        87.6   
124        maison 

In [14]:
df2v2['Date_réception_DPE'].count()

11624

In [15]:
def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        normalized = expand_address(address)  
        return "; ".join(normalized)  
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

df2v2['Adresse_Normalisee'] = df2v2['Adresse_(BAN)'].apply(normalize_address)

print(df2v2[['Date_réception_DPE', 'Adresse_(BAN)', 'Adresse_Normalisee']].head())

    Date_réception_DPE                                      Adresse_(BAN)  \
103         2022-04-28                      Le Petit Boulay 44850 Mouzeil   
124         2022-02-22               Rue du Belem 44470 Thouaré-sur-Loire   
418         2022-01-25    Allée de l’espérance 44270 Machecoul-Saint-Même   
613         2022-12-22       9 Rue Auguste-pasgrimaud 44170 La Grigonnais   
814         2022-05-30  34bis Rue du Clos Martin 44550 Montoir-de-Bret...   

                                    Adresse_Normalisee  
103                      le petit boulay 44850 mouzeil  
124  rue du belem 44470 thouare-sur-loire; rue du b...  
418  allee de l'esperance 44270 machecoul-saint-mem...  
613  9 rue auguste-pasgrimaud 44170 la grigonnais; ...  
814  34bis rue du clos martin 44550 montoir-de-bret...  


### VALEURS FONCIERES

In [16]:

vf = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/ValeursFoncieres/valeursfoncieres-2022.txt",
    sep="|",  
    encoding="utf-8",
    low_memory=False
)

In [17]:
print(vf[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune']].dtypes)


No voie         float64
Type de voie     object
Voie             object
Code postal     float64
Commune          object
dtype: object


In [18]:
# Convertir explicitement toutes les colonnes en chaînes
vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Type de voie'].fillna('').astype(str) + " " + \
                vf['Voie'].fillna('').astype(str) + ", " + \
                vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Commune'].fillna('').astype(str)

vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

print(vf[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune', 'Adresse']].head())


   No voie Type de voie           Voie  Code postal            Commune  \
0     13.0          RUE  DE LA LIBERTE       1000.0    BOURG-EN-BRESSE   
1      NaN          NaN   CHAMP COCHET       1480.0          SAVIGNEUX   
2      NaN          NaN   CHAMP COCHET       1480.0          SAVIGNEUX   
3     98.0          RTE   DE LA DOMBES       1480.0          SAVIGNEUX   
4      NaN          NaN    AUX PIERRES       1480.0  MESSIMY SUR SAONE   

                                      Adresse  
0  13 RUE DE LA LIBERTE, 1000 BOURG-EN-BRESSE  
1                CHAMP COCHET, 1480 SAVIGNEUX  
2                CHAMP COCHET, 1480 SAVIGNEUX  
3         98 RTE DE LA DOMBES, 1480 SAVIGNEUX  
4         AUX PIERRES, 1480 MESSIMY SUR SAONE  


In [19]:

def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        
        normalized = expand_address(address)
        return "; ".join(normalized)  
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

vf['Adresse_Normalisee'] = vf['Adresse'].apply(normalize_address)


print(vf[['Adresse', 'Adresse_Normalisee']].head())


                                      Adresse  \
0  13 RUE DE LA LIBERTE, 1000 BOURG-EN-BRESSE   
1                CHAMP COCHET, 1480 SAVIGNEUX   
2                CHAMP COCHET, 1480 SAVIGNEUX   
3         98 RTE DE LA DOMBES, 1480 SAVIGNEUX   
4         AUX PIERRES, 1480 MESSIMY SUR SAONE   

                                  Adresse_Normalisee  
0  13 rue de la liberte 1000 bourg-en-bresse; 13 ...  
1                        champ cochet 1480 savigneux  
2                        champ cochet 1480 savigneux  
3               98 route de la dombes 1480 savigneux  
4                 aux pierres 1480 messimy sur saone  


In [20]:
vf44 = vf[vf['Code postal'].notna()]
vf44 = vf44[vf44['Code postal'].astype(str).str.startswith('44')].copy()
print(vf44.head())

        Identifiant de document  Reference document  1 Articles CGI  \
101601                      NaN                 NaN             NaN   
101603                      NaN                 NaN             NaN   
101604                      NaN                 NaN             NaN   
101605                      NaN                 NaN             NaN   
101606                      NaN                 NaN             NaN   

        2 Articles CGI  3 Articles CGI  4 Articles CGI  5 Articles CGI  \
101601             NaN             NaN             NaN             NaN   
101603             NaN             NaN             NaN             NaN   
101604             NaN             NaN             NaN             NaN   
101605             NaN             NaN             NaN             NaN   
101606             NaN             NaN             NaN             NaN   

        No disposition Date mutation Nature mutation  ... Code type local  \
101601               1    04/01/2022           Vent

## **MATCHING AVEC PARSING**

FONCTION PARSING

In [21]:
from postal.parser import parse_address

def structured_normalization(address):
    """ Applique parse_address pour structurer l'adresse normalisée. """
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        parsed = parse_address(address)
        parsed_dict = {component: value for value, component in parsed}
        
        # Construire une adresse bien formatée
        normalized = f"{parsed_dict.get('house_number', '')} {parsed_dict.get('road', '')}, " \
                     f"{parsed_dict.get('postcode', '')} {parsed_dict.get('city', '')}"
        
        return normalized.strip()
    
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

In [22]:

dfv2['Adresse_Normalisee'] = dfv2['Adresse_Normalisee'].apply(structured_normalization)
df2v2['Adresse_Normalisee'] = df2v2['Adresse_Normalisee'].apply(structured_normalization)
vf44['Adresse_Normalisee'] = vf44['Adresse_Normalisee'].apply(structured_normalization)

In [23]:
print(len(dfv2))
print(len(df2v2))

64490
11624


In [24]:
print(f"Adresses uniques dans dfv2 : {dfv2['Adresse_Normalisee'].nunique()}")
print(f"Adresses uniques dans df2v2 : {df2v2['Adresse_Normalisee'].nunique()}")
print(f"Adresses uniques dans vf44 : {vf44['Adresse_Normalisee'].nunique()}")

Adresses uniques dans dfv2 : 20879
Adresses uniques dans df2v2 : 1890
Adresses uniques dans vf44 : 26559


In [25]:
print(f"Nombre d'adresses dupliquées dans dfv2 (logements existants) : {dfv2 ['Adresse_Normalisee'].value_counts().ge(2).sum()}")
print(f"Nombre d'adresses dupliquées dans df2v2 (logements neufs) : {df2v2 ['Adresse_Normalisee'].value_counts().ge(2).sum()}")
print(f"Nombre d'adresses dupliquées dans vf44 (valeurs foncières) : {vf44 ['Adresse_Normalisee'].value_counts().ge(2).sum()}")

Nombre d'adresses dupliquées dans dfv2 (logements existants) : 5783
Nombre d'adresses dupliquées dans df2v2 (logements neufs) : 552
Nombre d'adresses dupliquées dans vf44 (valeurs foncières) : 14989


TRAVAIL SUR LES DOUBLONS

In [ ]:

def calculate_duplicate_percentage(df, address_column):
    """
    On calcule le pourcentage d'adresses en doublon + on en fait une liste
    """
    total_count = len(df)
    duplicate_counts = df[address_column].value_counts()
    duplicate_addresses = duplicate_counts[duplicate_counts > 1].index.tolist()
    duplicate_count = len(duplicate_addresses)
    duplicate_percentage = (duplicate_count / total_count) * 100 if total_count > 0 else 0
    
    return duplicate_percentage, duplicate_addresses

# On applique la fonction aux logements existants
duplicate_percentage, duplicate_addresses = calculate_duplicate_percentage(dfv2, 'Adresse_Normalisee')

print(f"Pourcentage d'adresses en doublon : {duplicate_percentage:.2f}%")

liste = duplicate_addresses
print("Liste des adresses en doublon :")
print(liste)

Pourcentage d'adresses en doublon : 8.97%
Liste des adresses en doublon :
['44600 saintnazaire, 44600 saint nazaire', '44800 saintherblain, 44800 saint herblain', '44500 la bauleescoublac, 44500 la baule escoublac', '44230 saintsebastiensurloire, 44230 saint sebastien sur loire', '44980 saintelucesurloire, 44980 sainte luce sur loire', '44250 saintbrevinlespins, 44250 saint brevin les pins', '5 avenue robert chasteland, 44700 orvault', '44550 montoirdebretagne, 44550 montoir de bretagne', '44240 la chapellesurerdre, 44240 la chapelle sur erdre', '44430 le lorouxbottereau, 44430 le loroux bottereau', '44360 saintetiennedemontluc, 44360 saint etienne de montluc', '44290 guemenepenfao, 44290 guemene-penfao', '32 route de la joneliere, 44300 nantes', '24 rue blaise pascal, 44300 nantes', '44640 saintjeandeboiseau, 44640 saint jean de boiseau', '44150 ancenissaintgereon, 44150 ancenis saint', '44310 saintphilbertdegrandlieu, 44310 saint philbert de grand lieu', '44450 saintjuliendeconcelles